# Descritores dominantes e não dominantes

Ler um atributo por meio de uma instância devolve o atributo desta instância. O fallback é o atributo da classe.

Escrever/sobrescrever o valor de um atributo por meio de uma instância afeta apenas esta instância, sem fallback para a classe.

A assimetria está na implementação do método `__set__`.

In [1]:
from types import NoneType


def cls_name(obj):
    cls = type(obj)
    if cls is type:
        return obj
    return cls.__name__.split('.')[-1]

def display(obj):
    cls = type(obj)
    if cls is type:
        return f'<class {obj.__name__}>'
    elif cls in {NoneType, int}:
        return repr(obj)
    return f'<{cls_name(obj)} object>'

def print_args(name, *args):
    pseudo_args = ', '.join(display(x) for x in args)
    print(f'-> {cls_name(args[0])}.__{name}__({pseudo_args})')

class Overriding:  # <1>
    """a.k.a. data descriptor or enforced descriptor"""

    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)  # <2>

    def __set__(self, instance, value):
        print_args('set', self, instance, value)


class OverridingNoGet:  # <3>
    """an overriding descriptor without ``__get__``"""

    def __set__(self, instance, value):
        print_args('set', self, instance, value)


class NonOverriding:  # <4>
    """a.k.a. non-data or shadowable descriptor"""

    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)


class Managed:  # <5>
    over = Overriding()
    over_no_get = OverridingNoGet()
    non_over = NonOverriding()

    def spam(self):  # <6>
        print('-> Managed.spam({})'.format(display(self)))

**Descritor dominante** é aquele que implementa o dunder set. No caso, a classe `Overriding`. Este método, nesta classe, intercepta a escrita no atributo de instância gerenciada.

Propriedades também são descritores dominantes, pois
- `property` é uma classe;
- tem o método especial `set` (além de get e delete).

In [2]:
obj = Managed()
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [3]:
Managed.over

-> Overriding.__get__(<Overriding object>, None, <class Managed>)


In [4]:
obj['bla'] = 19
obj

TypeError: 'Managed' object does not support item assignment

In [6]:
obj.over = 19
obj.over

-> Overriding.__set__(<Overriding object>, <Managed object>, 19)
-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)
